In [1]:
import os
HOME = os.getcwd()
print(HOME)

c:\Users\Lenovo\Documents\Hardware Projects\SmartTraffic


## Install YOLOv8

In [2]:
%pip install ultralytics==8.3.19

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.19  Python-3.11.9 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i5-13500H)
Setup complete  (16 CPUs, 15.7 GB RAM, 156.8/700.7 GB disk)


## Install Roboflow Supervision

In [3]:
%pip install supervision[assets]==0.24.0
display.clear_output()

import supervision as sv
print("supervision.__version__:", sv.__version__)

supervision.__version__: 0.24.0


In [4]:
print(dir(sv))

['BackgroundOverlayAnnotator', 'BaseDataset', 'BlurAnnotator', 'BoundingBoxAnnotator', 'BoxAnnotator', 'BoxCornerAnnotator', 'ByteTrack', 'CSVSink', 'CircleAnnotator', 'ClassificationDataset', 'Classifications', 'Color', 'ColorAnnotator', 'ColorLookup', 'ColorPalette', 'ConfusionMatrix', 'CropAnnotator', 'DetectionDataset', 'Detections', 'DetectionsSmoother', 'DotAnnotator', 'EdgeAnnotator', 'EllipseAnnotator', 'FPSMonitor', 'HaloAnnotator', 'HeatMapAnnotator', 'IconAnnotator', 'ImageSink', 'InferenceSlicer', 'JSONSink', 'KeyPoints', 'LMM', 'LabelAnnotator', 'LineZone', 'LineZoneAnnotator', 'LineZoneAnnotatorMulticlass', 'MaskAnnotator', 'MeanAveragePrecision', 'OrientedBoxAnnotator', 'OverlapFilter', 'PercentageBarAnnotator', 'PixelateAnnotator', 'Point', 'PolygonAnnotator', 'PolygonZone', 'PolygonZoneAnnotator', 'Position', 'Rect', 'RichLabelAnnotator', 'RoundBoxAnnotator', 'TraceAnnotator', 'TriangleAnnotator', 'VertexAnnotator', 'VertexLabelAnnotator', 'VideoInfo', 'VideoSink', '__

## Render IRL video via cam

In [5]:
from supervision.assets import download_assets, VideoAssets

# SOURCE_VIDEO_PATH = download_assets(VideoAssets.VEHICLES)
# SOURCE_VIDEO_PATH= '/content/sample.mp4'



import cv2

def webcam_frame_generator():
    cap = cv2.VideoCapture(0)  # 0 is usually the default webcam
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        yield frame
    cap.release()

generator = webcam_frame_generator()





## Load pre-trained YOLOv8 model

In [6]:
from ultralytics import YOLO

model = YOLO("yolov8x.pt")

## Predict and annotate single frame

In [7]:
CLASS_NAMES_DICT = model.model.names

SELECTED_CLASS_NAMES = ['car', 'motorcycle', 'bus', 'truck']

# class ids matching the class names we have chosen
SELECTED_CLASS_IDS = [
    {value: key for key, value in CLASS_NAMES_DICT.items()}[class_name]
    for class_name
    in SELECTED_CLASS_NAMES
]

In [8]:
model.model.names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [9]:
SELECTED_CLASS_IDS

[2, 3, 5, 7]

In [10]:
import numpy as np
import cv2


## Predict and annotate whole video

In [11]:
import cv2

cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
cap.release()

print(f"Width: {width}, Height: {height}, FPS: {fps}")


Width: 640, Height: 480, FPS: 30.0


In [12]:
# settings
LINE_START = sv.Point(50, 336)  # 50 pixels from left, 70% down the frame (0.7*480 ≈ 336)
LINE_END   = sv.Point(590, 336) # 50 pixels from right edge (640-50), same vertical position


TARGET_VIDEO_PATH = f"{HOME}/result.mp4"

In [ ]:
import cv2
import numpy as np
import supervision as sv


# Create BYTETracker instance
byte_tracker = sv.ByteTrack(
    track_activation_threshold=0.25,
    lost_track_buffer=30,
    minimum_matching_threshold=0.8,
    frame_rate=30,
    minimum_consecutive_frames=3)
byte_tracker.reset()

# Define a custom webcam frame generator using OpenCV
def webcam_frame_generator():
    cap = cv2.VideoCapture(0)  # Use 0 for your default webcam
    if not cap.isOpened():
        print("Error: Unable to access the webcam.")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        yield frame
    cap.release()

# Use the custom webcam generator instead of a file-based generator
generator = webcam_frame_generator()

# Define the LineZone (ensure LINE_START and LINE_END are set for your camera resolution)
line_zone = sv.LineZone(start=LINE_START, end=LINE_END)

# Create annotators
box_annotator = sv.BoxAnnotator(thickness=4)
label_annotator = sv.LabelAnnotator(text_thickness=2, text_scale=1.5, text_color=sv.Color.BLACK)
trace_annotator = sv.TraceAnnotator(thickness=4, trace_length=50)
line_zone_annotator = sv.LineZoneAnnotator(thickness=4, text_thickness=4, text_scale=2)

# --- Define the frame processing callback ---
def callback(frame: np.ndarray, index: int) -> np.ndarray:
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    # Only consider class ids from SELECTED_CLASS_IDS defined elsewhere
    detections = detections[np.isin(detections.class_id, SELECTED_CLASS_IDS)]
    # Update tracker with detections
    detections = byte_tracker.update_with_detections(detections)
    labels = [
        f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}"
        for confidence, class_id, tracker_id
        in zip(detections.confidence, detections.class_id, detections.tracker_id)
    ]
    annotated_frame = frame.copy()
    annotated_frame = trace_annotator.annotate(scene=annotated_frame, detections=detections)
    annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)
    
    # Update line counter and annotate the line zone
    line_zone.trigger(detections)
    return line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)

# --- Process the webcam stream in real time ---
frame_index = 0
for frame in generator:
    processed_frame = callback(frame, frame_index)
    frame_index += 1
    cv2.imshow("Camera Feed", processed_frame)
    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


In [ ]:
# import os
# os.startfile(os.path.dirname(TARGET_VIDEO_PATH))
